In [1]:
#Importing the model

from keras.models import model_from_json

json_file = open(r'C:\Users\Chandan\Desktop\Face Mask Detection\Models\model.json','r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights(r'C:\Users\Chandan\Desktop\Face Mask Detection\Models\model.h5')

Using TensorFlow backend.
C:\Users\Chandan\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Chandan\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Chandan\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Chandan\anaconda3\lib\site-packages\tensorflow\python\framework\d

In [2]:
loaded_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [3]:
# Importing library
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [4]:
#Importing Data
df = pd.read_csv(r'C:\Users\Chandan\Desktop\Face Mask Detection\Dataset\dataset.csv')
df

,Unnamed: 0,index,0,1,2,3,4,5,6,7,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,0,0,232,232,232,232,232,232,232,232,...,115,117,122,128,109,39,37,38,155,207
1,1,0,255,255,255,255,255,255,255,254,...,89,83,76,87,75,70,3,0,209,254
2,2,0,255,255,255,255,255,255,255,255,...,31,5,25,19,5,18,12,4,2,1
3,3,0,255,255,254,254,254,254,254,254,...,140,164,158,138,125,125,254,255,255,255
4,4,0,244,242,240,241,242,240,242,242,...,57,57,40,29,34,47,61,43,73,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1371,1371,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1372,1372,1,119,120,118,118,119,120,123,123,...,22,51,39,47,40,37,29,31,45,54
1373,1373,1,249,249,249,248,248,248,248,248,...,99,103,167,157,102,98,96,100,103,107
1374,1374,1,200,201,202,203,202,201,198,175,...,24,18,17,12,21,23,20,21,19,19


In [5]:
from sklearn.model_selection import train_test_split
train = df.iloc[:,2:]
label = df.iloc[:,1]

train_x, test_x, train_y, test_y = train_test_split(train, label, random_state = 2, test_size = 0.2)

In [6]:
# Normalization
train_x = train_x.astype('float32')
train_x /= 255
test_x = test_x.astype('float32')
test_x /= 255

In [7]:
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)
(train_y.shape, test_y.shape)

((1100, 2), (276, 2))

In [8]:
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
train_x = train_x.values.reshape(-1,100,100,1)
test_x = test_x.values.reshape(-1,100,100,1)
(train_x.shape, test_x.shape)

((1100, 100, 100, 1), (276, 100, 100, 1))

In [9]:
print(loaded_model.evaluate(test_x,test_y))


276/276 [==============================] - 293s 1s/step
[0.08887134643568509, 0.9710144996643066]


In [10]:
face_clf = cv2.CascadeClassifier(r'C:\Users\Chandan\Desktop\Face Mask Detection/haarcascade_frontalface_default.xml')

source = cv2.VideoCapture(0)

labels_dict = {0:'With Mask', 1:'Without Mask'}
color_dict = {0:(0,255,0), 1:(0,0,255)}

In [11]:
while(True):
    ret, img = source.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_clf.detectMultiScale(gray,1.1,4)

    for (x,y,w,h) in faces:
        face_img = gray[x:x+w ,y:y+h]
        resized = cv2.resize(face_img, (100,100))
        normalized = resized / 255.0
        reshaped = np.reshape(normalized, (1,100,100,1))
        result = loaded_model.predict(reshaped)

        label = np.argmax(result,axis = 1)[0]

        cv2.rectangle(img, (x,y), (x+w,y+h), color_dict[label], 2)
        cv2.rectangle(img, (x,y-40), (x+w,y), color_dict[label], -1)
        cv2.putText(img, labels_dict[label], (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)


    cv2.imshow('LIVE', img)
    key = cv2.waitKey(1)

    if(key == 27):
        break

cv2.destroyAllWindows()
source.release()